# IMPORT PACKAGES

In [1]:
from evolutionary_algorithm import EvolutionaryAlgorithm as ea
from sklearn import datasets
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

# LOAD DATA

In [2]:
missing_values = ['N/A','--','n/a','NaN','na']
data=pd.read_csv(r"C:\Users\Nirav\OneDrive\Desktop\ALzheimers\DATASET.csv",na_values=missing_values)
print("total no.of rows and columns:",data.shape)
print("total size:",data.size)

total no.of rows and columns: (373, 15)
total size: 5595


# PRE-PROCESS DATA

In [3]:
data.columns=data.columns.str.capitalize()

In [4]:
data["Ses"].fillna(data.groupby("Educ")["Ses"].transform("median"), inplace=True)
# imputing MMSE with median values
data['Mmse'].fillna((data['Mmse'].median()), inplace=True)

In [5]:
data.rename(columns = {'M/f':'Gender'}, inplace = True)  

In [6]:
from sklearn import preprocessing  
label_encoder = preprocessing.LabelEncoder() 
data['Gender']= label_encoder.fit_transform(data['Gender']) 
data['Gender'].unique()

array([1, 0])

In [7]:
def categorize_Mmse(Mmse):
  if  19<= Mmse <=23 :
    return "mild cognitive impairement"
  elif 10 <= Mmse <= 18:
    return "moderate cognitive impairement "
  elif 0< Mmse <= 9:
    return "severe cognitive impairement"
  elif 24 <= Mmse <= 30:
    return "Normal cognition"
  

data['Dementia_category_Mmse'] = data['Mmse'].apply(lambda x: categorize_Mmse(x))

In [8]:
def categorize_Cdr(Cdr):
  if  Cdr <=0 :
    return "Non-demented"
  elif 0.1 < Cdr <= 0.5:
    return "mild demented"
  elif 0.6 < Cdr <= 1:
    return "very mild demented"
  elif 1.1 < Cdr <= 2:
    return "moderate dementia"
  elif 2.1 < Cdr <= 3:
    return "Severe dementia"

data['Dementia_category_Cdr'] = data['Cdr'].apply(lambda x: categorize_Cdr(x))

In [9]:
data1=data.copy()
data1 = data1.drop(['Mri id', 'Hand','Subject id','Visit','Group','Mr delay'], axis=1) # Drop unnecessary 

In [10]:
data1["Dementia_category_Mmse"] = data1["Dementia_category_Mmse"].astype("|S")
data1["Dementia_category_Cdr"] = data1["Dementia_category_Cdr"].astype("|S")

In [11]:
from sklearn import preprocessing  
label_encoder = preprocessing.LabelEncoder() 
data1['Dementia_category_Mmse']= label_encoder.fit_transform(data1['Dementia_category_Mmse']) 
data1['Dementia_category_Mmse'].unique()


array([0, 1, 2, 3], dtype=int64)

In [12]:
label_encoder = preprocessing.LabelEncoder()
data1['Dementia_category_Cdr']= label_encoder.fit_transform(data1['Dementia_category_Cdr']) 
data1['Dementia_category_Cdr'].unique()

array([0, 1, 3, 2], dtype=int64)

# DATA UPSAMPLING

In [13]:
data_majority = data1[data1.Dementia_category_Cdr==0]
data_minority = data1[data1.Dementia_category_Cdr==1]
from sklearn.utils import resample
data_minority_upsampled = resample(data_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=180,    # to match majority class
                                 random_state=4) # reproducible results
 
# Combine majority class with upsampled minority class
data_upsampled = pd.concat([data_majority, data_minority_upsampled])
 
# Display new class counts
#data_upsampled.balance.value_counts()
data_upsampled['Dementia_category_Cdr'].value_counts()

0    206
1    180
Name: Dementia_category_Cdr, dtype: int64

In [14]:
data_majority = data1[data1.Dementia_category_Cdr==1]
data_minority1 = data1[data1.Dementia_category_Cdr==2]

from sklearn.utils import resample
data_minority_upsampled1 = resample(data_minority1, 
                                 replace=True,     # sample with replacement
                                 n_samples=150,    # to match majority class
                                 random_state=4) # reproducible results
 
# Combine majority class with upsampled minority class
data_upsampled = pd.concat([data_upsampled, data_minority_upsampled1])
 
# Display new class counts
#data_upsampled.balance.value_counts()
data_upsampled['Dementia_category_Cdr'].value_counts()

0    206
1    180
2    150
Name: Dementia_category_Cdr, dtype: int64

In [15]:
data_majority = data1[data1.Dementia_category_Cdr==1]
data_minority2 = data1[data1.Dementia_category_Cdr==3]

from sklearn.utils import resample
data_minority_upsampled1 = resample(data_minority2, 
                                 replace=True,     # sample with replacement
                                 n_samples=150,    # to match majority class
                                 random_state=4) # reproducible results
 
# Combine majority class with upsampled minority class
data_upsampled = pd.concat([data_upsampled, data_minority_upsampled1])
 
# Display new class counts
#data_upsampled.balance.value_counts()
data_upsampled['Dementia_category_Cdr'].value_counts()


0    206
1    180
3    150
2    150
Name: Dementia_category_Cdr, dtype: int64

In [16]:
data1=data_upsampled.copy()

# SUPPORT VECTOR MACHINE BEFORE TUNING

In [17]:
X=data1.iloc[:,1:-1]
y = pd.DataFrame(data1.iloc[:,-1])

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2,random_state=10)

In [19]:
from sklearn import svm

#Create a svm Classifier
clf1 = svm.SVC(kernel='rbf') 

#Train the model using the training sets
clf1.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf1.predict(X_test)

In [20]:
from sklearn.metrics import accuracy_score
SVC1=accuracy_score(y_test, y_pred)
print("Accuracy:",SVC1)

Accuracy: 0.2753623188405797


# EVOLUTIONARY ALGORITHM

In [21]:
objective_parameters = [
    {'name' : 'C',
     'bounds' : [0.1, 1, 10, 100, 1000],
     'type' : 'float'},
    {'name' : 'gamma',   
     'bounds' : [1, 0.1, 0.01, 0.001, 0.0001],
     'type' : 'float'},
]

In [22]:
# Define the objective function (with embedded predictive model)
def objective_function(args):

    clf = svm.SVC(**args)
    clf.fit(X_train, y_train)
    preds = clf.predict(X)

    return accuracy_score(y, preds) * -1 # Expects a value to be minimized


In [23]:
# Create instance of EA object
evo_algo = ea(function=objective_function, 
           parameters=objective_parameters)

# Run EA
evo_algo.run()

# Access best model parameters
evo_algo.best_parameters

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  5.49it/s]

Best parameters found: {'C': 0.840851501229314, 'gamma': 0.43478049417713327}
Best objective output = -0.9854227405247813

{'C': 0.840851501229314, 'gamma': 0.43478049417713327}

# SUPPORT VECTOR AFTER TUNING

In [ ]:
from sklearn import svm
clf2 = svm.SVC(kernel='rbf',C= 0.8353310732867345, gamma = 0.5018917022412928)
clf2.fit(X_train, y_train)
y_pred1 = clf2.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
SVC2=accuracy_score(y_test, y_pred1)
print("Accuracy:",SVC2)